In [30]:
!pip install tensorflow
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [31]:
symbol = "TSLA"
start_date = "2023-01-01"
end_date = "2023-07-31"

In [32]:
stock_data = yf.download(symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [33]:
data = stock_data['Adj Close'].values.reshape(-1, 1)

In [34]:
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

In [35]:
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i : i + n_steps, 0])
        y.append(data[i + n_steps, 0])
    return np.array(X), np.array(y)

In [36]:
n_steps = 10

In [37]:
X, y = prepare_data(data_scaled, n_steps)

In [38]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [39]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [40]:
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
5/5 [==============================] - 3s 10ms/step - loss: 0.3265
Epoch 2/50
5/5 [==============================] - 0s 8ms/step - loss: 0.2618
Epoch 3/50
5/5 [==============================] - 0s 7ms/step - loss: 0.1969
Epoch 4/50
5/5 [==============================] - 0s 8ms/step - loss: 0.1410
Epoch 5/50
5/5 [==============================] - 0s 8ms/step - loss: 0.0871
Epoch 6/50
5/5 [==============================] - 0s 7ms/step - loss: 0.0441
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0148
Epoch 8/50
5/5 [==============================] - 0s 8ms/step - loss: 0.0076
Epoch 9/50
5/5 [==============================] - 0s 7ms/step - loss: 0.0120
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0074
Epoch 11/50
5/5 [==============================] - 0s 7ms/step - loss: 0.0064
Epoch 12/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0066
Epoch 13/50
5/5 [==============================] - 0s 8ms/step - loss: 0

In [41]:
test_data = data_scaled[-n_steps:].reshape((1, n_steps, 1))
predicted_price = model.predict(test_data)

1/1 [==============================] - 0s 381ms/step


In [42]:
predicted_price = scaler.inverse_transform(predicted_price)
print(f"Predicted Price: {predicted_price[0, 0]}")

Predicted Price: 268.95843505859375
